## Tensorflow程序主要包括两个阶段: 1. 定义计算图中的所有计算; 2. 执行计算(会话)。

### 在tensorflow程序中，系统会自动维护一个默认的计算图

### 通过 tf.get_default_graph 函数来获取当前默认的计算图。

In [2]:
import tensorflow as tf

v = tf.get_variable(name="v", shape=[1], initializer=tf.zeros_initializer)

print(v.graph is tf.get_default_graph())

Instructions for updating:
Colocations handled automatically by placer.
True


### 生成计算图的方法

In [3]:
g = tf.Graph() # tf.Graph()函数生成计算图
# 在该计算图下定义变量及计算
with g.as_default():
    v = tf.get_variable(name="v", shape=[1], initializer=tf.zeros_initializer)
print(v.graph is g)

True


### 获取计算图中变量

In [6]:
with tf.Session(graph=g) as sess:
    sess.run(tf.global_variables_initializer())
    with tf.variable_scope("", reuse=True):
        print(sess.run(tf.get_variable("v")))

[0.]


### 将计算图运行在指定的设备

In [17]:
g1 = tf.Graph()
with g1.as_default():
    a = tf.constant([1., 2., 3.], name="a", dtype=tf.float32)
    b = 2 * a
with g1.device("/cpu:0"):
    result = a + b
with tf.Session(graph=g1) as sess:
    print(sess.run(result))
    print(result.graph is g1)

[3. 6. 9.]
True


### 在一个计算图中，可以通过集合(collection)来管理不同类别的资源

### tf.add_to_collection()将资源加入一个或多个集合中

### tf.get_collection()获取一个集合里面所有的资源

### 其中，"资源"指的是张量、变量或者队列资源等

### Tensorflow中自动维护的集合列表：

### tf.GraphKeys.VARIABLES  -->  所有变量

### tf.GraphKeys.TRAINABLE_VARIABLES  -->  可学习的变量（在定义变量时, 将参数trainable设置为True）

### tf.GraphKeys.SUMMARIES  -->  日志生成相关的张量

### tf.GraphKeys.QUEUE_RUNNERS  -->  处理输入的QueueRunner

### tf.GraphKeys.MOVING_AVERAGE_VARIABLES --> 所有计算了滑动平均值的变量

In [33]:
tf.get_collection(tf.GraphKeys.VARIABLES) #获取变量集合

<tf.Variable 'v:0' shape=(1,) dtype=float32_ref>

### collection常用于管理损失

In [34]:
x1 = tf.constant(1.0)
l1 = tf.nn.l2_loss(x1) # 第一部分损失
x2 = tf.constant([2.5, -0.3])
l2 = tf.nn.l2_loss(x2) # 第二部分损失
# 手动添加到集合
tf.add_to_collection("losses", l1)
tf.add_to_collection("losses", l2)
losses = tf.get_collection("losses")
# 计算总体损失
loss_total = tf.add_n(losses)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(losses))
    print(sess.run(loss_total))

[0.5, 3.17]
3.67
